# Import libraries and setup git root

In [23]:
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.cluster import KMeans
import torch
import numpy as np
import pandas as pd
import fastparquet ## make sure environment has fastparquet and NOT pyarrow
import itertools
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pybktree import BKTree
import time 
import ast


In [24]:
import os

def find_repo_root(start_path):
    """
    useful general function for finding the (first, closest) repo root so github file paths work the same on different machines 
    """
    current_path = os.path.abspath(start_path)
    
    while True:
        # Check for the existence of the .git directory or other indicators
        if os.path.isdir(os.path.join(current_path, '.git')) or \
           os.path.isfile(os.path.join(current_path, 'README.md')):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        
        # Stop if we reach the root directory
        if parent_path == current_path:
            break
        
        current_path = parent_path

    return None  # Return None if not found

root = find_repo_root(os.getcwd())
root = root.replace('\\', '/')
print(root)


c:/Users/fitsl/Documents/Programming/UVM Programming Classes/PoCS/pocs_project


In [25]:
df = pd.read_csv(f"{root}/Data/2020_trope_data/Scraped_Data/merged_NER_scraped_data.csv")

In [26]:
df['studio'].iloc[1]

"['TriStar Pictures', 'Tencent Pictures', 'Big Beach']"

# Import the comments data

In [27]:
df = pd.read_csv(F"{root}/Data/2020_trope_data/Scraped_Data/merged_NER_scraped_data.csv", index_col=0)

Now, we pick say 

In [28]:
df['reviews_extracted'].iloc[2]

'[\'ugh, his mind\', \'I guess you could say Paul Bettany wasa vision\', \'Here’s an equation for ya:male gaze + runtime exceeding 2 hours + slightly brown tinted colour grading = Academy Award for Best Picture, 2001\', \'his mind wasn’t even that beautiful\', "jennifer connelly is beautiful alright. i\'m not sure about his mind though", "How\'s this for a problem without a solution: who gives a shit", \'Somehow simultaneously the impressively dull, saccharine, sanded-of-any-and-all-edges prestige biopic that we\\\'ve been assaulted with for 2 decades nowanda "what ifThe Sixth Sensebut your brain didn\\\'t work right" disability gawking horror show filled with so many close-up push-ins on Jennifer Connolly looking in "wow, maybe he is still a real person in there" awe it\\\'s no wonder she secured the Oscar and not him. How life-affirming. Thank you, Ron Howard.\', \'moral of the story: don’t marry your professor\', \'He do be doin math tho\', \'"So what am I now, a spy?"Not only did I

In [29]:
df['reviews_extracted'].head(5)

0    ['To the creators of "P is for P-P-P-P SCARY!"...
1    ['therapy: expensivetom hanks as mr rogers bre...
2    ['ugh, his mind', 'I guess you could say Paul ...
3    ['Nobody understands the male friendship & bro...
4    ["Cute as a bug's rear!A heartwarming coming o...
Name: reviews_extracted, dtype: object

In [30]:
import enchant
d = enchant.Dict("en_US")
d.check("Helo")

False

In [31]:
from spellchecker import SpellChecker

# Initialize the SpellChecker with no language
spell = SpellChecker(language=None)

# Custom list of words to add to the dictionary
my_words = ["custom", "spell", "checker", "dictionary", "example", "words"]

# Add each custom word to the spell checker
for word in my_words:
    spell.word_frequency.add(word)

# List of words to check against the custom dictionary
words_to_check = ["spell", "checkr", "example", "wordz", "spelling"]
     
# Check each word
for word in words_to_check:
    if word in spell:
        print(f"The word '{word}' is recognized.")
    else:
        # If the word is not recognized, suggest a correction
        correction = spell.correction(word)
        print(f"The word '{word}' is not recognized. Did you mean '{correction}'?")


The word 'spell' is recognized.
The word 'checkr' is not recognized. Did you mean 'checker'?
The word 'example' is recognized.
The word 'wordz' is not recognized. Did you mean 'words'?
The word 'spelling' is not recognized. Did you mean 'None'?


In [32]:
def combine_adjacent_blocks(text, block_pattern, print_pattern):
    # This pattern looks for occurrences of the block followed by the same block, and replaces them with a single block
    result = re.sub(rf'({block_pattern} {block_pattern})', print_pattern, text)
    return result

block = r'\[Actor\]'  # You can replace '[Actor]' with any other block if necessary
input_text = "I thought [Actor] [Actor] and [Actor] [Actor] were amazing in the movie! [Actor] [Actor] performance was also stellar."
output_text = combine_adjacent_blocks(input_text, block, "[Actor]")
print(output_text)

I thought [Actor] and [Actor] were amazing in the movie! [Actor] performance was also stellar.


In [33]:
def split_names(names_list):
    split_name_list = []
    names_list = ast.literal_eval(names_list)  # Assuming names_list is a string representation of a list
    for name in names_list:
        parts = name.split()
        if len(parts) > 1:
            # Only take the first and last names
            first_name = parts[0].lower()
            last_name = parts[-1].lower()
            split_name_list.append(first_name)
            split_name_list.append(last_name)
        else:
            # If only one name is provided (e.g., a single first name), just add it
            split_name_list.append(parts[0].lower())
    
    return split_name_list


def split_review(review):
    # Replace slashes and periods with spaces to treat them as word delimiters
    review_cleaned = re.sub(r'[/.]', ' ', review)  # Replace slashes and periods with spaces
    
    # Remove possessive 's or ’s (e.g., John's -> John, Marielle’s -> Marielle)
    review_cleaned = re.sub(r"[’']s\b", '', review_cleaned)  # Handles both straight and curly apostrophes
    
    # Remove unwanted characters (non-alphanumeric, except for common punctuation)
    review_cleaned = re.sub(r'[^a-zA-Z0-9?!.;, ]', '', review_cleaned)
    
    # Split the cleaned string into words (spaces are already used as delimiters)
    words = review_cleaned.split()
    
    return words

In [34]:
def extract_words_from_reviews(review_list):
    all_words = set()
    for review in review_list:
        words = split_review(review)  # Split the review into words
        all_words.update(map(lambda w: w.lower(), words))  # Normalize words to lowercase
    return all_words

# Function to add fuzzy matches for names against the vocabulary
def add_fuzzy_matches(names, vocabulary, placeholder, threshold=90):
    fuzzy_match_map = defaultdict(list)

    for name in names:
        matches = process.extract(name, vocabulary, limit=100)  # Get top 100 fuzzy matches for each name
        for match, score in matches:
            if score >= threshold:  # Only accept matches above the threshold
                fuzzy_match_map[match].append(placeholder)

    return fuzzy_match_map

In [35]:
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define the title and the review
title = "A Beautiful Mind"
review = """
There is a turning point in A Beautiful Mind. If you have seen it or know anything about its subject matter you know the point I am speaking of. At that point the film goes from being a pretty average movie, and then flips it on its head and becomes undeniably great. Ranking the Oscars Best Pictures is a very interesting topic, and many people have different views on it. The second half of this film is full of brilliant performances, not just from Crowe but also an especially harrowing performance by Connelly…
"""

# Function to replace title in the review using fuzzy matching
def replace_title_in_review(title, review, threshold=80):
    # We use a regular expression to find potential title occurrences (case-insensitive)
    words = review.split()
    
    # Check each word to see if it fuzzy matches the title, and replace it if it does
    modified_review = []
    i = 0
    while i < len(words):
        word = words[i]
        
        # Check fuzzy match of the word with the title, allow for punctuation in between
        sentence_chunk = " ".join(words[i:i+len(title.split())])  # Grab chunk based on title length
        
        # If the chunk is similar enough to the title, replace it
        score = fuzz.ratio(title.lower(), sentence_chunk.lower())
        if score >= threshold:
            modified_review.append("[TITLE]")
            i += len(sentence_chunk.split())  # Skip the matched chunk length
        else:
            modified_review.append(word)
            i += 1

    return " ".join(modified_review)

# Replace the title with "TITLE"
modified_review = replace_title_in_review(title, review)

print("Original Review:")
print(review)
print("\nModified Review:")
print(modified_review)


Original Review:

There is a turning point in A Beautiful Mind. If you have seen it or know anything about its subject matter you know the point I am speaking of. At that point the film goes from being a pretty average movie, and then flips it on its head and becomes undeniably great. Ranking the Oscars Best Pictures is a very interesting topic, and many people have different views on it. The second half of this film is full of brilliant performances, not just from Crowe but also an especially harrowing performance by Connelly…


Modified Review:
There is a turning point in [TITLE] If you have seen it or know anything about its subject matter you know the point I am speaking of. At that point the film goes from being a pretty average movie, and then flips it on its head and becomes undeniably great. Ranking the Oscars Best Pictures is a very interesting topic, and many people have different views on it. The second half of this film is full of brilliant performances, not just from Crowe

In [36]:
def replace_actors_with_placeholder(review_list, actor_list, director_list, title, role_list):


    # Split the review into words
    review_list = ast.literal_eval(review_list)
    actor_list = split_names(actor_list)
    director_list = split_names(director_list)
    role_list = split_names(role_list)


    matchers = [
        ('[DIRECTOR]', director_list),
        ('[ROLE]', role_list),
        ('[ACTOR]', actor_list),
    ]

    updated_reviews = []
    for review in review_list:
        updated_words = [] 

        for word in split_review(review):
            replaced = False

            # Try to match the word with each matcher in order
            for placeholder, match_list in matchers:
                match = process.extractOne(word, match_list)
                if match and match[1] >= 97:  # Threshold for a valid match
                    updated_words.append(placeholder) # Replace the word if met
                    replaced = True
                    break 
            if not replaced:
                updated_words.append(word)  # No match, keep the word as is
        
        # Join the updated words back into a string
        updated_review = " ".join(updated_words)
        updated_review = replace_title_in_review(title=title, review=updated_review)
        updated_review = combine_adjacent_blocks(updated_review,  r'\[ACTOR\]', "[ACTOR]")
        updated_review = combine_adjacent_blocks(updated_review,  r'\[ROLE\]', "[ROLE]")
        updated_review = combine_adjacent_blocks(updated_review,  r'\[DIRECTOR\]', "[DIRECTOR]")


        print(review, "\n", updated_review, "\n", f"{'--'*50}")
        updated_reviews.append(updated_review)
    print(f"parsed {title}")
    return updated_reviews

In [37]:
import nltk
from nltk.corpus import words

# Download the 'words' corpus if you don't have it
# nltk.download('words')

# Load the list of common words in English
english_words = set(words.words())

def filter_names_from_dict(word_list):
    # Filter out any words that are in the dictionary
    # print("filterint")
    return [word for word in word_list if word.lower() not in english_words]

# Example usage
words = ["John", "dog", "Mary", "house", "Paris"]
filtered_names = filter_names_from_dict(words)

print(filtered_names)


['John', 'Paris']


Need to fix:
* logic that is not matching stuff like Rodgers'
* Seems to miss at the end maybe...
* matching on roles that arent real or that should be considered -- maybe strip all the ones that have uncredited, and strip all ordinals like first, etc

In [44]:
from fuzzywuzzy import process
from collections import defaultdict
import ast

# Function to extract all words from reviews (vocabulary)
def extract_words_from_reviews(review_list):
    all_words = set()
    for review in review_list:
        words = split_review(review)  # Split the review into words
        all_words.update(map(lambda w: w.lower(), words))  # Normalize words to lowercase
    return all_words

# Function to add fuzzy matches for names against the vocabulary
def add_fuzzy_matches(names, vocabulary, placeholder, threshold=90, verbose=None):
    fuzzy_match_map = defaultdict(list)
    for name in names:
        matches = process.extract(name, vocabulary, limit=50)  # Get the top 30 fuzzy matches for each name
        if verbose==2: print(name, " : ", matches)
        for match, score in matches:
            if score >= threshold:  # Only accept matches above the threshold
                fuzzy_match_map[match.lower()].append(placeholder)
            else:
                break  # Stop matching once a score below the threshold is found
    if verbose: print(fuzzy_match_map)
    return fuzzy_match_map

# Main function to replace actors, directors, roles in reviews with placeholders
def replace_actors_with_placeholder(review_list, actor_list, director_list, title, role_list, verbose=None):
    # Parse the review list (assuming it's a string representation of a list)
    review_list = ast.literal_eval(review_list)

    
    # Split the names into a list of individual names (case insensitive)
    actor_list = split_names(actor_list)
    director_list = split_names(director_list)
    role_list = filter_names_from_dict(split_names(role_list))
    title = str(title)

    # Extract vocabulary (words in all reviews)
    vocabulary = extract_words_from_reviews(review_list)

    # Generate fuzzy matches for actors, directors, and roles
    fuzzy_match_map = defaultdict(list)
    fuzzy_match_map.update(
        add_fuzzy_matches(actor_list, vocabulary, '[ACTOR]', threshold=95, verbose=verbose))
    fuzzy_match_map.update(
        add_fuzzy_matches(director_list, vocabulary, '[DIRECTOR]', threshold=95, verbose=verbose))
    fuzzy_match_map.update(
        add_fuzzy_matches(role_list, vocabulary, '[ROLE]', threshold=95, verbose=verbose))
    
    # Initialize list to store updated reviews
    updated_reviews = []

    # Iterate over each review in the review list
    for review in review_list:
        updated_words = []

        # Iterate through each word in the review
        for word in split_review(review):
            normalized_word = word.lower()

            # Check if the word matches any fuzzy match in the map
            if normalized_word in fuzzy_match_map:
                updated_words.append(fuzzy_match_map[normalized_word][0])  # Use the first match (placeholder)
            else:
                updated_words.append(word)  # No match, keep the original word

        # Join the updated words back into a single string
        updated_review = " ".join(updated_words)
        
        # Replace the title in the review
        try:
            updated_review = replace_title_in_review(title=title, review=updated_review)
        except Exception as e:
            print(e, review)

        # Combine adjacent blocks of placeholders (if needed)
        updated_review = combine_adjacent_blocks(updated_review, r'\[ACTOR\]', "[ACTOR]")
        updated_review = combine_adjacent_blocks(updated_review, r'\[ROLE\]', "[ROLE]")
        updated_review = combine_adjacent_blocks(updated_review, r'\[DIRECTOR\]', "[DIRECTOR]")

        # Add the updated review to the list
        updated_reviews.append(updated_review)
        if verbose: print(f"{review}\n{updated_review}'\n'{'--' * 50}")

    print(f"Parsed {title}")
    return updated_reviews




In [39]:
# chunksize = 50
# output_file = f'{root}/Data/2020_trope_data/Scraped_Data/NER_cleaned.csv'
# from datetime import datetime


# # Iterate over the DataFrame in chunks
# for start in range(0, len(df), chunksize):
#     chunk = df.iloc[start:start + chunksize]
#     current_time = datetime.now()
#     print(f"Started {start} to {start + chunksize}  at  " + current_time.strftime("%Y-%m-%d %H:%M:%S"))
    
#     # Apply the function to each row in the chunk
#     chunk['NER_reviews'] = chunk.apply(
#         lambda row: replace_actors_with_placeholder(
#             review_list=row['reviews_extracted'],
#             actor_list=row['actors'],
#             role_list=row['roles'],
#             title=row['NameIMDB'],
#             director_list=row['letterboxd_directors']
#         ),
#         axis=1
#     )
#     current_time = datetime.now()
#     print(f"finished {start} to {start + chunksize}  at  " + current_time.strftime("%Y-%m-%d %H:%M:%S"))
#     chunk.to_csv(output_file, mode='a', header=not bool(start), index=False)



In [46]:
df_small = df.iloc[10612:10620].copy()
df_small['Matched_Actors'] = df_small.apply(
      lambda row: replace_actors_with_placeholder(
            review_list = row['reviews_extracted'],
            actor_list = row['actors'],
            role_list = row['roles'],
            title= row['NameIMDB'],
            director_list = row['letterboxd_directors']
            )
            , axis=1)




Parsed Zygote
Parsed Zyuden Sentai Kyoryuger: Gaburincho of Music
Parsed nan
Parsed A Nightmare on Elm Street
Parsed A Quiet Place
Parsed Adam
Parsed Air Force One
Parsed nan


In [ ]:
split_review("Heller’s filmmaking is so warm and inviting. She has the ability to take character’s who on the surface feel like supporting characters in real life and makes them protagonists (obviously referring to Llyod, not Mr. Rogers.) That said, while she held my attention with his character, the performance from Matthew Rhys felt pretty cartoonish and stereotypical. Same with Chris Cooper, he didn’t really feel believable till the last 20 minutes. The editing was interesting and I appreciate the stylistic choices…")

['Heller',
 'filmmaking',
 'is',
 'so',
 'warm',
 'and',
 'inviting',
 'She',
 'has',
 'the',
 'ability',
 'to',
 'take',
 'character',
 'who',
 'on',
 'the',
 'surface',
 'feel',
 'like',
 'supporting',
 'characters',
 'in',
 'real',
 'life',
 'and',
 'makes',
 'them',
 'protagonists',
 'obviously',
 'referring',
 'to',
 'Llyod,',
 'not',
 'Mr',
 'Rogers',
 'That',
 'said,',
 'while',
 'she',
 'held',
 'my',
 'attention',
 'with',
 'his',
 'character,',
 'the',
 'performance',
 'from',
 'Matthew',
 'Rhys',
 'felt',
 'pretty',
 'cartoonish',
 'and',
 'stereotypical',
 'Same',
 'with',
 'Chris',
 'Cooper,',
 'he',
 'didnt',
 'really',
 'feel',
 'believable',
 'till',
 'the',
 'last',
 '20',
 'minutes',
 'The',
 'editing',
 'was',
 'interesting',
 'and',
 'I',
 'appreciate',
 'the',
 'stylistic',
 'choices']

In [ ]:
from transformers import pipeline

# Load a pretrained BERT model for POS tagging
pos_tagger = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

text = "Tom Hanks wasn't even acting here, he was just always like this."

# Get POS tags
pos_tags = pos_tagger(text)
print(pos_tags)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER', 'score': 0.9978801, 'index': 1, 'word': 'Tom', 'start': 0, 'end': 3}, {'entity': 'I-PER', 'score': 0.9929189, 'index': 2, 'word': 'Hank', 'start': 4, 'end': 8}, {'entity': 'I-PER', 'score': 0.95965266, 'index': 3, 'word': '##s', 'start': 8, 'end': 9}]


In [ ]:
df_small['Matched_Actors'].iloc[0]

['therapy expensivetom [ACTOR] as [ROLE] breaking the forth wall asking how you deal with anger free or the cost of a cinema ticket',
 'There a moment in this where [ACTOR] [ROLE] looks right at the camera and into your soul for a solid 20 seconds, and it simultaneously beautiful and terrifying, as if maybe this is the last thing youll ever see Rest of the movie is pretty good too',
 '[ACTOR] wasnt even acting here, he just always like this',
 '[DIRECTOR] filmmaking is so warm and inviting She has the ability to take character who on the surface feel like supporting characters in real life and makes them protagonists obviously referring to Llyod, not [ROLE] That said, while she held my attention with his character, the performance from [ACTOR] felt pretty cartoonish and stereotypical Same with [ACTOR] he didnt really feel believable till the last 20 minutes The editing was interesting and I appreciate the stylistic choices',
 'Saw this a couple days ago and cant stop thinking about how

In [ ]:
-- break

SyntaxError: invalid syntax (2555494183.py, line 1)

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

# List of actors (correct spellings)
actor_list = ['Russell Crowe', 'Jennifer Connelly', 'Ed Harris', 'Paul Bettany', 'Christopher Plummer', 'Josh Lucas', 'Adam Goldberg']

# Function to tokenize the review text (split by spaces and remove punctuation)
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# Function to generate token-based n-grams from the review
def generate_ngrams(tokens, n):
    return [tokens[i:i+n] for i in range(len(tokens) - n + 1)]

# Function to replace actor names in the review with '[ACTOR]' and track indexes
def match_actor_names_with_ngrams(
            review_list = None,
            actor_list = None,
            role_list = None,
            title= None,
            director_list = None,
        threshold=80):
    
    max_actor_length = max(len(actor.split()) for actor in actor_list)
    mod_reviews = []

    for review in review_list:

        original_review = review  # Keep the original review for matching
        matched_indexes = []
        
        # Tokenize the review into words
        tokens = tokenize(review)

        # Find the length of the longest actor name for n-gram generation

        # Generate n-grams from the tokens (based on word length)
        ngrams = generate_ngrams(tokens, max_actor_length)

        # Create a list to store the replacements without modifying the string yet
        replacements = []

        # Iterate over each n-gram
        for ngram in ngrams:
            ngram_str = ' '.join(ngram)  # Convert n-gram back to a string
            # Use fuzzy matching between the n-gram and each actor name
            for actor in actor_list:
                score = fuzz.partial_ratio(ngram_str, actor.lower())  # Use partial_ratio for fuzzy matching
                
                if score >= threshold:
                    # Find the start and end indices of the matched n-gram in the original review
                    pattern = re.escape(ngram_str)
                    matches = list(re.finditer(r'\b' + pattern + r'\b', original_review.lower()))

                    for match in matches:
                        matched_indexes.append((match.start(), match.end(), actor))
                        # Store the replacement, adjusting for the shift caused by earlier replacements
                        replacements.append((match.start(), match.end(), '[ACTOR]'))

        # Sort replacements by their start index (from earliest to latest), to avoid index shifting
        replacements.sort()

        # Build the final modified review by replacing the matched names with '[ACTOR]'
        modified_review = []
        prev_end = 0
        for start, end, replacement in replacements:
            modified_review.append(original_review[prev_end:start])  # Add text before the match
            modified_review.append(replacement)  # Add the '[ACTOR]' replacement
            prev_end = end  # Update the previous end to the end of the match

        # Append the remaining part of the original review after the last replacement
        modified_review.append(original_review[prev_end:])

        # Join the list into a single string for the final modified review
        modified_review = ''.join(modified_review)
        mod_reviews.append(modified_review)

    print(f"parsed {title}")
    return mod_reviews

# Example review with a misspelled actor name
review_list = ["I guess you could say Paul Betanny was a vision in the movie with my man Russell Crowe lol.", "jennifer connelly is beautiful alright. i\'m not sure about his mind though"]

# Replace actor names and track indexes
modified_review = match_actor_names_with_ngrams(review_list, actor_list, threshold=80)

# Display results
print("Original Review:", review_list)
print("Modified Review:", modified_review)


parsed None
Original Review: ['I guess you could say Paul Betanny was a vision in the movie with my man Russell Crowe lol.', "jennifer connelly is beautiful alright. i'm not sure about his mind though"]
Modified Review: ['I guess you could say [ACTOR] was a vision in the movie with my man [ACTOR] lol.', "[ACTOR][ACTOR] beautiful alright. i'm not sure about his mind though"]


In [ ]:
   
    # return matched_actors

# Apply the function to the DataFrame
df_small = df.iloc[1:2].copy()
df_small['Matched Actors'] = df_small.apply(
      lambda row: match_actor_names_with_ngrams(
            review_list = row['reviews_extracted'],
            actor_list = row['actors'],
            role_list = row['roles'],
            title= row['NameIMDB'],
            director_list = row['letterboxd_directors']
            )
            , axis=1)

# Show the DataFrame with matched actors
# df_small['reviews_extracted'].iloc[2]


KeyboardInterrupt: 

In [ ]:
df_small['reviews_extracted'].iloc[2]

IndexError: single positional indexer is out-of-bounds

In [ ]:
df_small

,letterboxd_search,url,actors,roles,studio,reviews_extracted,review_stars,review_dates,letterboxd_directors,letterboxd_year,letterboxd_rating,letterboxd_genres,NameIMDB,IMDB_rating,IMDB_ID,letter_USD_Budget,letter_US_Gross,letter_WW_Gross,Matched Actors
0,https://letterboxd.com/search/ABCs+of+Death+2+...,https://letterboxd.com/film/abcs-of-death-2/,"['Eric Jacobus', 'Julian Barratt', 'Ian Virgo'...","['Assassin (Segment ""Amateur"")', 'Peter Toland...","['Drafthouse Films', 'Timpson Films']","['To the creators of ""P is for P-P-P-P SCARY!""...","[6, 2, 7, 4, 9, 7, 5, 6, 4, 7, 6, 4, 6, 7, 2, ...","['14 Feb 2024', '16 Apr 2022', '17 Oct 2018', ...","['Rodney Ascher', 'Bill Plympton', 'Erik Matti...",2014,5.48,['Horror'],ABCs of Death 2,5.4,tt2926810,NaN,NaN,7171.0,[
1,https://letterboxd.com/search/A+Beautiful+Day+...,https://letterboxd.com/film/a-beautiful-day-in...,"['Matthew Rhys', 'Tom Hanks', 'Chris Cooper', ...","['Lloyd Vogel', 'Fred Rogers', 'Jerry Vogel', ...","['TriStar Pictures', 'Tencent Pictures', 'Big ...",['therapy: expensivetom hanks as mr rogers bre...,"[6, 7, 7, 6, 7, 8, 8, 8, 8, 6, 6, 8, 8, 8, 8, ...","['13 Oct 2019', '27 Nov 2019', '22 Jan 2020', ...",['Marielle Heller'],2019,7.20,"['History', 'Drama']",A Beautiful Day in the Neighborhood,0.0,tt3224458,25000000.0,61704055.0,67925733.0,[
2,https://letterboxd.com/search/A+Beautiful+Mind...,https://letterboxd.com/film/a-beautiful-mind/,"['Russell Crowe', 'Jennifer Connelly', 'Ed Har...","['John Nash', 'Alicia Nash', 'William Parcher'...","['Universal Pictures', 'DreamWorks Pictures', ...","['ugh, his mind', 'I guess you could say Paul ...","[8, 10, 4, 6, 4, 3, 2, 8, 3, 6, 6, 4, 8, 4, 7,...","['12 Mar 2018', '24 Sep 2020', '15 Jul 2018', ...",['Ron Howard'],2001,7.68,"['Drama', 'Romance']",A Beautiful Mind,8.2,tt0268978,58000000.0,170742341.0,316791257.0,[
3,https://letterboxd.com/search/A+Better+Tomorro...,https://letterboxd.com/film/a-better-tomorrow/,"['Ti Lung', 'Chow Yun-fat', 'Leslie Cheung', '...","['Sung Chi Ho', ""Mark 'Gor' Lee"", 'Kit', 'Jack...","['Cinema City', 'Film Workshop']",['Nobody understands the male friendship & bro...,"[8, 10, 8, 10, 8, 10, 9, 8, 8, 7, 9, 8, 9, 10,...","['30 Apr 2021', '22 May 2021', '07 Mar 2021', ...",['John Woo'],1986,7.76,"['Drama', 'Crime', 'Action']",A Better Tomorrow,7.6,tt0092263,NaN,NaN,4999517.0,[
4,https://letterboxd.com/search/A+Birder's+Guide...,https://letterboxd.com/film/a-birders-guide-to...,"['Kodi Smit-McPhee', 'James Le Gros', 'Daniela...","['David Portnoy', 'Donald Portnoy', 'Juliana S...","['There We Go Films', 'dreamFly Productions', ...","[""Cute as a bug's rear!A heartwarming coming o...","[3, 6, 7, 7, 6, 7, 6, 6, 6, 6, 5, 10, 6, 8, 6,...","['28 May 2014', '09 Jul 2014', '09 Jun 2014', ...",['Rob Meyer'],2013,6.38,['Comedy'],A Birder's Guide to Everything,6.3,tt1582465,NaN,NaN,NaN,[


In [ ]:
--break

In [ ]:
#### Schema idea from chatgpt #### 

# Example sets of actors, directors, and films
actors_list = ['Leonardo DiCaprio', 'Meryl Streep', 'Brad Pitt']  # You can expand this list
directors_list = ['Quentin Tarantino', 'Steven Spielberg', 'Martin Scorsese']  # Expand as needed
films_list = ['Inception', 'The Matrix', 'Pulp Fiction', 'Forrest Gump']  # Expand as needed

def replace_named_entities_with_keywords(text, actors, directors, films):
    # First, replace known actors, directors, and films with tags
    for actor in actors:
        text = re.sub(r'\b' + re.escape(actor) + r'\b', '[ACTOR]', text)
        
    for director in directors:
        text = re.sub(r'\b' + re.escape(director) + r'\b', '[DIRECTOR]', text)
        
    for film in films:
        text = re.sub(r'\b' + re.escape(film) + r'\b', '[FILM]', text)
    
    return text

def replace_named_entities_with_bert(text, nlp_ner):
    # Use BERT NER pipeline to extract named entities
    entities = nlp_ner(text)
    
    # Replace each recognized entity with a placeholder (except for those already tagged by the keyword pass)
    for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
        entity_text = text[entity['start']:entity['end']]
        
        # General NER replacement (skip already tagged entities like [ACTOR], [DIRECTOR], [FILM])
        if '[ACTOR]' not in entity_text and '[DIRECTOR]' not in entity_text and '[FILM]' not in entity_text:
            text = text[:entity['start']] + f"[{entity['entity']}]"+ text[entity['end']:]
    
    # Clean up tags by replacing 'B-' and 'I-' labels into a single tag
    text = re.sub(r'\[B-([A-Z]+)\](\s*\[I-\1\])+', r'[\1]', text)
    text = re.sub(r'\[[BI]-(\w+)\]', r'[\1]', text)

    return text

def process_review(text, nlp_ner):
    # Step 1: Keyword-based pass to tag actors, directors, and films
    text = replace_named_entities_with_keywords(text, actors_list, directors_list, films_list)
    
    # Step 2: Apply BERT NER pass to identify and tag other entities
    text = replace_named_entities_with_bert(text, nlp_ner)
    
    return text


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Example predefined list of proper nouns (celebrities)
known_names = ["Russell Crowe", "Tom Hanks", "Brad Pitt", "Angelina Jolie", "Robert Downey Jr."]

def normalize_name(name, known_names, threshold=80):
    """
    Function to check if the given name is similar to any known proper nouns.
    If the similarity is above a given threshold, returns the normalized name.
    """
    # Get the closest match using fuzzywuzzy's process.extractOne
    match = process.extractOne(name, known_names, scorer=fuzz.ratio)
    
    if match and match[1] >= threshold:
        return match[0]  # Return the normalized name
    return name  # If no match, return the original name

# Test with some names
names_to_check = ["Russel Crow", "Russell Crow", "Tom Hanks", "Tomm Hanks", "Angelina Jollee"]

for name in names_to_check:
    normalized_name = normalize_name(name, known_names)
    print(f"Original: {name} -> Normalized: {normalized_name}")


Original: Russel Crow -> Normalized: Russell Crowe
Original: Russell Crow -> Normalized: Russell Crowe
Original: Tom Hanks -> Normalized: Tom Hanks
Original: Tomm Hanks -> Normalized: Tom Hanks
Original: Angelina Jollee -> Normalized: Angelina Jolie


In [ ]:
Other thoughts


# from transformers import pipeline
# from fuzzywuzzy import process
# from collections import defaultdict
# import ast

# # Initialize the BERT-based NER pipeline
# ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# # Function to extract all words from reviews (vocabulary)
# def extract_words_from_reviews(review_list):
#     all_words = set()
#     for review in review_list:
#         words = split_review(review)  # Split the review into words
#         all_words.update(map(lambda w: w.lower(), words))  # Normalize words to lowercase
#     return all_words

# # Function to add fuzzy matches for names against the vocabulary
# def add_fuzzy_matches(names, vocabulary, placeholder, threshold=90, verbose=None):
#     fuzzy_match_map = defaultdict(list)
#     for name in names:
#         matches = process.extract(name, vocabulary, limit=50)  # Get the top 50 fuzzy matches for each name
#         if verbose == 2: 
#             print(name, " : ", matches)
#         for match, score in matches:
#             if score >= threshold:  # Only accept matches above the threshold
#                 fuzzy_match_map[match.lower()].append(placeholder)
#             else:
#                 break  # Stop matching once a score below the threshold is found
#     if verbose: print(fuzzy_match_map)
#     return fuzzy_match_map

# # Function to replace actors, directors, roles in reviews with placeholders using BERT NER and fuzzy matching
# def replace_actors_with_placeholder(review_list, actor_list, director_list, title, role_list):
#     # Parse the review list (assuming it's a string representation of a list)
#     review_list = ast.literal_eval(review_list)

#     # Split the names into a list of individual names (case insensitive)
#     actor_list = split_names(actor_list)
#     director_list = split_names(director_list)
#     role_list = split_names(role_list)

#     # Extract vocabulary (words in all reviews)
#     vocabulary = extract_words_from_reviews(review_list)

#     # Generate fuzzy match maps for actors, directors, and roles
#     fuzzy_match_map = defaultdict(list)
#     fuzzy_match_map.update(add_fuzzy_matches(actor_list, vocabulary, '[ACTOR]', threshold=90))
#     fuzzy_match_map.update(add_fuzzy_matches(director_list, vocabulary, '[DIRECTOR]', threshold=90))
#     fuzzy_match_map.update(add_fuzzy_matches(role_list, vocabulary, '[ROLE]', threshold=90))

#     # Initialize list to store updated reviews
#     updated_reviews = []

#     # Iterate over each review in the review list
#     for review in review_list:
#         # Step 1: Use BERT to get named entities
#         tokens = ner_pipeline(review)  # Use BERT to get named entities
#         updated_words = []

#         # Create a dictionary to store recognized entities and their corresponding placeholders
#         entity_map = defaultdict(list)

#         for entity in tokens:
#             # Extract the entity and its label
#             entity_word = entity.get('word', '').lower()

#             # Check if the entity corresponds to a name in the fuzzy match map
#             if entity_word in fuzzy_match_map:
#                 entity_map[entity_word].append(fuzzy_match_map[entity_word][0])  # Use the first match (placeholder)

#         # Step 2: Iterate through the review's words and replace recognized entities with placeholders
#         for word in split_review(review):
#             normalized_word = word.lower()

#             # Check if the word is recognized by fuzzy matching
#             if normalized_word in entity_map:
#                 updated_words.append(entity_map[normalized_word][0])  # Use the first placeholder
#             else:
#                 updated_words.append(word)  # No match, keep the original word

#         # Join the updated words back into a single string
#         updated_review = " ".join(updated_words)

#         # Replace the title in the review (as per original code)
#         updated_review = replace_title_in_review(title=title, review=updated_review)

#         # Combine adjacent blocks of placeholders (if needed)
#         updated_review = combine_adjacent_blocks(updated_review, r'\[ACTOR\]', "[ACTOR]")
#         updated_review = combine_adjacent_blocks(updated_review, r'\[ROLE\]', "[ROLE]")
#         updated_review = combine_adjacent_blocks(updated_review, r'\[DIRECTOR\]', "[DIRECTOR]")

#         # Add the updated review to the list
#         updated_reviews.append(updated_review)
#         print(f"{review}\n{updated_review}\n{'--' * 50}")

#     print(f"Parsed {title}")
#     return updated_reviews
